In [5]:
import argparse
import os
import random
import sys
sys.path.append('../')
from poker_env_multi import PokerEnvMulti
from agents.random_policy import RandomActions
from ray.rllib.algorithms.ppo import PPOConfig

import ray
from ray import air, tune
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.algorithms.pg import (
    PG,
    PGConfig,
    PGTF2Policy,
    PGTF1Policy,
    PGTorchPolicy,
)

from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env

In [6]:
def env_creator(config):
    env = PokerEnvMulti(config)
    return env

register_env("poker", lambda config: env_creator(config))

In [8]:
def select_policy(agent_id, episode, **kwargs):
    #if agent_id == "player_0":
    #    return "learned"
    #else:
    #    return random.choice(["always_same", "beat_last"])
    return "random"

config = (
    PPOConfig()
    .rollouts(
        num_rollout_workers=0,
        num_envs_per_worker=1,
        rollout_fragment_length=10,
    )
    .training(
        train_batch_size=128,
        gamma=0.99,
    )
    .multi_agent(
        policies={
            "random": PolicySpec(policy_class=RandomActions),
            #"learned": PolicySpec(
            #    config={
            #        "model": {"use_lstm": use_lstm},
            #        "framework": args.framework,
            #    }
            #),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=[],
    )
)

trainer = config.build(env="poker")


2022-10-31 16:09:39,027	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.
2022-10-31 16:09:39,028	WARNING ppo.py:350 -- `train_batch_size` (128) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=10)! Auto-adjusting `rollout_fragment_length` to 128.
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ray/_private/ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.1/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-10-31 16:09:42,388	INFO worker.py:1518 -- Started a local Ray instance.
2022-10-31 16:09:44,620	WARNING env.py

AttributeError: 'PokerEnvMulti' object has no attribute 'observation_space'